In [10]:
from pathlib import Path

import tensorflow as tf
from tensorflow.keras.models import load_model

In [12]:
DATA_PATH = Path("~/data/split-v3-random-nongray").expanduser() 
BASE_PATH = Path("~/models/model-from-scratch-dropout-earlier/").expanduser()

In [8]:
model = load_model(str(BASE_PATH / "export"))

In [11]:
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# tflite_model = converter.convert()

# with (BASE_PATH / "export-tflite.tflite").open('wb') as f:
#     f.write(tflite_model)

## Post-training quantization

In [25]:
import numpy as np

In [26]:
mean = tf.Variable([[[0.74136317, 0.73086095, 0.74473]]])
IMAGE_SIZE = 336

In [27]:
# A generator that provides a representative dataset
def representative_data_gen(sample_size=100):
    sample_paths = np.random.choice(list((DATA_PATH / "train").glob("*/*.jpeg")), 
                                    size=sample_size)
    
    for img_path in sample_paths:
        image = tf.io.read_file(str(img_path))
        image = tf.io.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])
        image = tf.cast(image / 128., tf.float32) - mean
        image = tf.expand_dims(image, 0)
        yield [image]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
# This enables quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.int8]
# This ensures that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# And this sets the representative dataset so we can quantize the activations
converter.representative_dataset = representative_data_gen
tflite_model = converter.convert()

with (BASE_PATH / "export-tflite_quant.tflite").open('wb') as f:
    f.write(tflite_model)